In [ ]:
import sys, os
import pandas as pd

import numpy as np
import pandas as pd

import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join("../..")))

In [ ]:
from src.feature_extraction.preprocess import (
    obfuscate_and_filter_tokens,
    preprocess_text,
)


dataset_path = "../../data/mixed_dataset.csv"


data = pd.read_csv(dataset_path, na_filter=False)


data["Body"] = data["Body"].apply(preprocess_text)
data["Subject"] = data["Subject"].apply(preprocess_text)

data["Body"] = data["Body"].apply(obfuscate_and_filter_tokens)
data["Subject"] = data["Subject"].apply(obfuscate_and_filter_tokens)


texts = data["Body"] + data["Subject"]


labels = data["isPhishing"]


tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")


tokenizer.fit_on_texts(texts)


word_index = tokenizer.word_index


sequences = tokenizer.texts_to_sequences(texts)


padded_sequences = pad_sequences(sequences, maxlen=100)
print(len(word_index))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=4444
)

# Model definition
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=100))
model.add(Conv1D(filters=128, kernel_size=5, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy", tensorflow.keras.metrics.F1Score],
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, y_test),
    verbose=1,
)

In [ ]:
print(f"Max Validation Accuracy: {max(history.history["val_accuracy"])}")
print(f"Average Validation Accuracy: {sum(history.history["val_accuracy"])/len(history.history["val_accuracy"])}")

print(f"Max Validation F1 Score: {max(history.history["val_f1_score"])}")
print(f"Average Validation F1 Score: {sum(history.history["val_f1_score"])/len(history.history["val_f1_score"])}")

In [ ]:
import matplotlib.pyplot as plt

# Plot the training and validation loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

# Plot the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.plot(history.history["f1_score"], label="Training F1")
plt.plot(history.history["val_f1_score"], label="Validation F1")
plt.title("Training and Validation F1")
plt.xlabel("Epochs")
plt.ylabel("F1 Score")
plt.legend()

plt.show()

In [ ]:
loss, accuracy, f1 = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {accuracy}")
print(f"Test f1: {f1}")